In [ ]:
import pyspark
import gdown
from pyspark.sql import SparkSession
from pyspark.sql.types import*
from pyspark.sql.functions import date_format, col, count, when, regexp_extract, avg, var_pop, format_number, min, max, first, last, count_distinct, asc, sum
from datetime import date
from pyspark.ml.feature import StringIndexer, VectorAssembler, PCA, MinMaxScaler
from pyspark.ml.regression import LinearRegression
from pyspark.sql.types import IntegerType
from os.path import join
from pyspark.sql.window import Window


#LEITURA ESCRITA: _LE

spark= SparkSession.builder.getOrCreate()

#videos-stats
url_LE='https://drive.google.com/file/d/1qoYIh2XjG_Gqy5dA6fVwAVqZW0PuIf7R/view?usp=drive_link'

# Extração do arquivo pela URL
file_id_LE = url_LE.split('/')[-2]

# Download do arquivo
output_file_LE = 'videos-stats.csv'
gdown.download(id=file_id_LE, output=output_file_LE, quiet=False)

# Leitura do arquivo CSV usando spark
df = spark.read.csv(output_file_LE)

#Visualize os primeiros 8 registro do arquivo
df.show(8)

#Visualize o esquema do arquivo
df.printSchema()

#Leia novamente o arquivo inferindo o esquema e visualize o esquema novamente

df = spark.read.csv(output_file_LE, header=True, inferSchema=True)
df.printSchema()


#Salve o arquivo como 'videos-parquet' no formato parquet e adicione o cabeçalho nos dados

df.write.option('header','true').mode("overwrite").parquet('videos-parquet')


#Leia e visualize o arquivo 'videos-parquet' com cabeçalho nos dados;

df = spark.read.parquet('videos-parquet')
df.show()

#Salve o arquivo do exec. anterior como tabela chamada ‘tb_videos’ no banco de dados default do spark catalog

df.write.option('header','true').mode("overwrite").saveAsTable('tb_videos')

#Liste as tabelas do spark catalog para verificar a tabela

spark.catalog.listTables()

#Utilize o spark SQL para ler a tabela ‘tb_videos’

tab_df = spark.sql('select * from tb_videos')
tab_df.show()

#Leia o arquivo ‘comments.csv' inferindo o esquema e visualize

url_LE2='https://drive.google.com/file/d/1NqytO5N-riQGo9-puSFsDnaWtRzKA5dv/view?usp=drive_link'

# Extração do arquivo pela URL
file_id_LE2 = url_LE2.split('/')[-2]

# Download do arquivo
output_file_LE2 = 'comments.csv'
gdown.download(id=file_id_LE2, output=output_file_LE2, quiet=False)

#Leitura do arquivo
df = spark.read.csv(output_file_LE2, header=True, inferSchema=True)
df.printSchema()

#Salve o arquivo como ‘comments-parquet' no formato parquet e adicione o cabeçalho nos dados

df.write.option('header','true').mode("overwrite").parquet('comments-parquet')


#TRATAMENTO : _TR

#Leia o arquivo ‘videos-stats.csv'  no dataframe 'df_video' com cabeçalho e inferindo o esquema
df_video_url_TR='https://drive.google.com/file/d/1XAefeuunm6q7o0OEdF--NzEPXTmIG9i5/view?usp=drive_link'
# Extração do arquivo pela URL
file_id_TR = df_video_url_TR.split('/')[-2]
# Download do arquivo
output_file_TR = 'videos-stats.csv'
gdown.download(id=file_id_TR, output=output_file_TR, quiet=False)
#Leitura do arquivo inferindo o esquema
df_video_TR= spark.read.csv(output_file_TR, header=True, inferSchema=True)
df_video_TR.printSchema()

#Altere os valores nulos dos campos 'Likes', 'Comments' e 'Views' para o valor 0

df_video_TR = df_video_TR.fillna({'Likes': 0, 'Comments': 0, 'Views': 0})
df_video_TR.show()

#Leia o arquivo ‘comments.csv' no dataframe 'df_comentario' com cabeçalho e inferindo o esquema

df_comentario_url_TR='https://drive.google.com/file/d/1rW-4ufh6UqafMAeRrH1MVypHkS75lfFM/view?usp=drive_link'

# Extração do arquivo pela URL
file_id_TR = df_comentario_url_TR.split('/')[-2]

# Download do arquivo
output_file_TR = 'comments.csv'
gdown.download(id=file_id_TR, output=output_file_TR, quiet=False)

#Leitura do arquivo
df_comentario_TR = spark.read.csv(output_file_TR, header=True, inferSchema=True)
df_comentario_TR.printSchema()

#Calcule a quantidade de registros do df_video e df_comentario

df_video_contagem_TR  = df_video_TR.count()
df_comentario_contagem_TR = df_comentario_TR.count()

print("Quantidade de registros em df_video:", df_video_contagem_TR)
print("Quantidade de registros em df_comentario:", df_comentario_contagem_TR)

#Remova os registros do df_video e df_comentario quem possuem o campo 'Video ID' nulos e calcule novamente a quantidade de registros

df_video_TR = df_video_TR.dropna(subset=['Video ID'])
df_comentario_TR = df_comentario_TR.dropna(subset=['Video ID'])

df_video_contagem_TR  = df_video_TR.count()
df_comentario_contagem_TR = df_comentario_TR.count()

print("Quantidade de registros sem nulos em df_video:", df_video_contagem_TR)
print("Quantidade de registros sem nulos em df_comentario:", df_comentario_contagem_TR)

#Remova os registros apenas do df_video quem possuem o campo 'Video ID' duplicados

df_video_TR = df_video_TR.dropDuplicates(['Video ID'])
df_video_contagem_TR  = df_video_TR.count()

print("Quantidade de registros sem duplicados em df_video:", df_video_contagem_TR)

#Converta os campos Likes, Comments e Views para 'int' no dataframe df_video

df_video_TR = df_video_TR.withColumn("Likes", df_video_TR["Likes"].cast(IntegerType()))

#Converta os campos Likes e Sentiment para 'int' no dataframe df_comentario, além disso, altere o nome do campo Likes para 'Likes Comment'

df_comentario_TR = df_comentario_TR.withColumn("Likes", df_comentario_TR["Likes"].cast(IntegerType()))
df_comentario_TR = df_comentario_TR.withColumn("Sentiment", df_comentario_TR["Sentiment"].cast(IntegerType()))
df_comentario_TR = df_comentario_TR.withColumnRenamed("Likes", "Likes Comment")

#Crie o campo 'Interaction' no dataframe df_video, com a soma dos campos Likes, Comments e Views

df_video_TR = df_video_TR.withColumn("Interaction", df_video_TR["Likes"] + df_video_TR["Comments"] + df_video_TR["Views"])
print(df_video_TR.show())

#Converta os campos 'Published At' para 'date' no dataframe df_video

df_video_TR = df_video_TR.withColumn("Published At", df_video_TR["Published At"].cast(DateType()))

#Crie o campo 'Year' no dataframe df_video, extraindo apenas o ano do campo 'Published At'

df_video_TR = df_video_TR.withColumn("Year", date_format(df_video_TR["Published At"], "yyyy"))

#Mescle os dados df_comentario no dataframe df_video em relação ao campo Video ID e crie o dataframe df_join_video_comments

mescla_video_comentario_TR = join("df_video_TR", "df_comentario_TR")
df_join_video_comments_TR = df_video_TR.join(df_comentario_TR, on="Video ID", how="left")

#Leia o arquivo ‘videos-stats.csv'  no dataframe 'df_video' com cabeçalho e inferindo o esquema
df_us_videos_url_TR='https://drive.google.com/file/d/1NmT0RoxSv9GSJ08qDIMdeRkwbsaXzJE_/view?usp=drive_link'
# Extração do arquivo pela URL
file_id_TR = df_us_videos_url_TR.split('/')[-2]
# Download do arquivo
output_file_TR = 'USvideos.csv.csv'
gdown.download(id=file_id_TR, output=output_file_TR, quiet=False)
#Leitura do arquivo inferindo o esquema
df_us_videos_TR= spark.read.csv(output_file_TR, header=True, inferSchema=True)
df_us_videos_TR.printSchema()

#Mescle os dados df_us_videos no dataframe df_video em relação ao campo Title e crie e visualize o dataframe df_join_video_usvideos

mescla_us_videos_video_TR = join("df_video_TR", "df_us_videos_TR")
df_join_video_usvideos_TR = df_video_TR.join(df_us_videos_TR, on="Title", how="left")
df_join_video_usvideos_TR.show()

#Verifique a quantidade de campos nulos em todos os campos do dataframe df_video

df_video_nulos_TR = df_video_TR.select([count(when(col(c).isNull(), c)).alias(c) for c in df_video_TR.columns])
df_video_nulos_TR.show()

#Remova a coluna '_c0' e salve o dataframe df_video como 'videos-tratados-parquet' no formato parquet e adicione o cabeçalho nos dados

df_video_TR = df_video_TR.drop("_c0")
df_video_TR.write.option('header','true').mode("overwrite").parquet('videos-tratados-parquet')

#Remova a coluna '_c0' e salve o dataframe df_join_video_comments como 'videos-comments-tratados-parquet' no formato parquet e adicione o cabeçalho nos dados

df_join_video_comments_TR = df_join_video_comments_TR.drop("_c0")
df_join_video_comments_TR.write.option('header','true').mode("overwrite").parquet('videos-comments-tratados-parquet')

#PREPARAÇÃO : _PR

#Leia o arquivo ‘videos-tratados.snappy.parquet' no dataframe 'df_video'

df_video_PR = videos-tratados.snappy.parquet

#Adicione a coluna 'Month' com o valor do mês da coluna "Published At"

df_video_PR = df_video_PR.withColumn('Month', date_format(col('Published At'), 'MM').cast('int')) #Ajustado de 'mmmm' para 'mm' e colocado como inteiro

#Adicione a coluna "Keyword Index" com a transformação da coluna 'keyword' para valores numéricos

indexer_PR = StringIndexer(inputCol='Keyword', outputCol='Keyword Index')
df_video_PR = indexer.fit(df_video_PR).transform(df_video_PR)

#Crie um vetor chamado "Features" com os campos: "Likes", "Views", "Year", "Month", "Keyword Index" e transforme o dataframe df_video com o VectorAssembler, lembrando que o vetor só aceita campos do tipo numérico

df_video_PR = df_video_PR.withColumn("Year", df_video_PR["Year"].cast(IntegerType())) ## Convertido para inteiro
df_video_PR = df_video_PR.na.drop()
montar_vetor_PR = VectorAssembler(inputCols=['Likes', 'Views', 'Year', 'Month', 'Keyword Index'],
                               outputCol='Features')
df_video_PR = montar_vetor_PR.transform(df_video_PR)

#Adicione a coluna "Features Normal" com os dados normalizados da coluna Features, lembrando que para normalizar a coluna não pode conter valores nulos

df_video_PR = df_video_PR.na.drop()

scaler_PR= MinMaxScaler(inputCol='Features', outputCol='Features Normal')
modelo_scaler_PR = scaler_PR.fit(df_video_PR)
df_video_PR = modelo_scaler_PR.transform(df_video_PR)

#Adicione a coluna "Features PCA" com a redução de 5 características para 1, utilizando o modelo PCA

pca_PR = PCA(k=1, inputCol='Features Normal', outputCol='Features PCA')
modelo_pca_PR = pca_PR.fit(df_video_PR)
df_video_PR = modelo_pca_PR.transform(df_video_PR)

#Separe o dataframe df_video em 2 conjuntos: 80% para treinamento e 20% para teste

df_treino_PR, df_teste_PR = df_video_PR.randomSplit([0.8, 0.2], seed=42)

#Crie um modelo de regressão linear para estimar o valor do campo "Comments", utilizando a "Features Normal" e avalie o modelo

regressão_linear_PR = LinearRegression(featuresCol='Features Normal', labelCol='Comments')
modelo_rl_PR = regressão_linear_PR.fit(df_treino_PR)

avaliacao_rl_PR = modelo_rl_PR.evaluate(df_teste_PR)
print("RMSE(Erro quadrático médio da raiz):", avaliacao_rl_PR.rootMeanSquaredError)
print("R2(Coeficiente de determinação):", avaliacao_rl_PR.r2)

#Salve o dataframe df_video como 'videos-preparados-parquet' no formato parquet

df_video_PR.write.option('header','true').mode("overwrite").parquet('videos-preparados.snappy.parquet')


#AGREGAÇÃO : _AG

#Leia o arquivo 'videos-preparados.snappy.parquet' no dataframe 'df_video'

df_video_url_AG='https://drive.google.com/file/d/145CE3eezLNunzvr8aofldQq4wTlBaCsR/view?usp=sharing'
# Extração do arquivo pela URL
file_id_AG = "145CE3eezLNunzvr8aofldQq4wTlBaCsR"
# Download do arquivo
output_file_AG = 'videos-preparados.snappy.parquet'
gdown.download(id=file_id_AG, output=output_file_AG, quiet=False, fuzzy=True)
#Leitura do arquivo inferindo o esquema
df_video_AG = spark.read.parquet(output_file_AG)

#Calcule a quantidade de registros para cada valor único da coluna "Keyword"

df_video_AG.groupBy("Keyword").count().show()

#Calcule a média da coluna "Interaction" para cada valor único da coluna 'Keyword'

df_video_AG.groupBy("Keyword").avg("Interaction").show()

#Calcule o valor máximo da coluna "Interaction" para cada valor único da coluna 'Keyword' e nomeie de 'Rank Interactions', em seguida ordene pela nova coluna em ordem decrescente

df_video_AG.groupBy("Keyword").max("Interaction").withColumnRenamed("max(Interaction)", "Rank Interactions").orderBy("Rank Interactions", ascending=False).show()

#Calcule a média e a variância da coluna 'Views' para cada valor único da coluna 'Keyword'

df_video_AG.groupBy("Keyword").agg(avg("Views"), var_pop("Views")).show()

#Calcule a média, o valor mínimo e o valor máximo de 'Views' para cada valor único da coluna 'Keyword', sem casas decimais

df_video_AG.groupBy("Keyword").agg(
    format_number(avg("Views"), 2).alias("avg_views"),
    format_number(min("Views"), 2).alias("min_views"),
    format_number(max("Views"), 2).alias("max_views")
).show()

#Mostre o primeiro e o último 'Published At' para cada valor único da coluna 'Keyword'

df_video_AG.groupBy("Keyword").agg(first("Published At"), last("Published At")).show()

#Conte todos os 'title' de forma normal e todos os únicos e verifique se há diferença

df_video_AG.groupBy("title").agg(
      count("title").alias("Contagem normal"),
      count_distinct("title").alias("Contagem unicos")
      ).show()

#Mostre a quantidade de registros ordenados por ano em ordem ascendente

df_video_AG.groupBy(df_video_AG["Published At"].substr(1, 4).alias("Ano")).count().orderBy("Ano").show()

#Mostre a quantidade de registros ordenados por ano e mês em ordem ascendente

df_video_AG.groupBy(df_video_AG["Published At"].substr(1, 4).alias("Ano"),
                 df_video_AG["Published At"].substr(6, 2).alias("Mes")).count().orderBy("Ano", "Mes").show()

#Calcule a média acumulativa de ‘Likes’ para cada ‘Keyword’ ao longo dos anos

df_video_AG = df_video_AG.withColumn("Ano", df_video_AG["Published At"].substr(1, 4))
window_spec_AG = Window.partitionBy("Keyword").orderBy("Ano").rowsBetween(Window.unboundedPreceding, Window.currentRow)
df_video_AG = df_video_AG.withColumn("Media Acumulada Likes", format_number(avg("Likes").over(window_spec_AG), 0))
df_video_AG.show()

#OTIMIZAÇÃO : _OT

#Leia o arquivo 'videos-preparados.snappy.parquet' no dataframe 'df_video'

df_video_url_OT='https://drive.google.com/file/d/145CE3eezLNunzvr8aofldQq4wTlBaCsR/view?usp=drive_link'
# Extração do arquivo pela URL
file_id_OT = df_video_url_OT.split('/')[-2]
# Download do arquivo
output_file_OT = 'videos-preparados.snappy.parquet'
gdown.download(id=file_id_OT, output=output_file_OT, quiet=False)
#Leitura do arquivo inferindo o esquema
df_video_OT = spark.read.parquet(output_file_OT)

#Leia o arquivo ‘video-comments-tratados.snappy.parquet' no dataframe 'df_comments

df_comments_url_OT='https://drive.google.com/file/d/1e3CYHGCNb2INUt-w0iMQM-9ZT5QPQ6eV/view?usp=drive_link'
# Extração do arquivo pela URL
file_id_OT = df_comments_url_OT.split('/')[-2]
# Download do arquivo
output_file_OT = 'video-comments-tratados.snappy.parquet'
gdown.download(id=file_id_OT, output=output_file_OT, quiet=False)
#Leitura do arquivo inferindo o esquema
df_comments_OT = spark.read.parquet(output_file_OT)

# Renomear colunas para evitar espaços e inconsistências
df_video_cleaned_OT = df_video_OT.withColumnRenamed("Video ID", "video_id")
df_comments_cleaned_OT = df_comments_OT.withColumnRenamed("Video ID", "video_id")

#Crie tabelas temporárias para ambos os dataframe

df_video_cleaned_OT.createOrReplaceTempView('video_table')
df_comments_cleaned_OT.createOrReplaceTempView('comments_table')

#Faça um join das tabelas criadas anteriormente utilizando o spark.sql no dataframe ‘join_video_comments

join_video_comments_OT = spark.sql("""
    SELECT v.*, c.*
    FROM video_table v
    JOIN comments_table c
    ON v.video_id = c.video_id
""")

join_video_comments_OT.show()

#Faça as mesmas etapas anteriores utilizando repartition e coalesce

# Renomeando a coluna 'Video ID' para 'video_id' no DataFrame df_video
df_video_cleaned_OT = df_video_OT.withColumnRenamed("Video ID", "video_id")
# Renomeando a coluna 'Video ID' para 'video_id' no DataFrame df_comments
df_comments_cleaned_OT = df_comments_OT.withColumnRenamed("Video ID", "video_id")

##  Realizar o JOIN com reparticionamento
df_video_repartition_OT = df_video_cleaned_OT.repartition(10)
df_comments_repartition_OT = df_comments_cleaned_OT.repartition(10)

# Criar tabelas temporárias novamente
df_video_repartition_OT.createOrReplaceTempView("video_table_repartition")
df_comments_repartition_OT.createOrReplaceTempView("comments_table_repartition")

#  JOIN com reparticionamento
join_video_comments_repartition_OT = spark.sql("""
    SELECT v.*, c.*
    FROM video_table_repartition v
    JOIN comments_table_repartition c
    ON v.video_id = c.video_id
""")

join_video_comments_repartition_OT.show()

#  Aplicar coalesce
df_video_coalesce_OT = df_video_cleaned_OT.coalesce(2)
df_comments_coalesce_OT = df_comments_cleaned_OT.coalesce(2)

#  Criar tabelas temporárias para os DataFrames coalesce
df_video_coalesce_OT.createOrReplaceTempView("video_table_coalesce")
df_comments_coalesce_OT.createOrReplaceTempView("comments_table_coalesce")

#  Realizar o JOIN utilizando as tabelas temporárias coalesce
join_video_comments_coalesce_OT = spark.sql("""
    SELECT v.*, c.*
    FROM video_table_coalesce v
    JOIN comments_table_coalesce c
    ON v.video_id = c.video_id
""")
join_video_comments_coalesce_OT.show()

#Utilize o explain para entender melhor as duas formas de realizar as etapas e refaça novamente as etapas anteriores, utilizando tudo que você já aprendeu para realizar o join e filter apenas com os dados necessários.

# Visualizar as formas de realização:

#Com repartition
join_video_comments_repartition_OT.explain()

#Com coalesce
join_video_comments_coalesce_OT.explain()

#Padronização dos dataframes

df_video_cleaned_OT = df_video_OT.select(
    *[col(c).alias(c.lower().replace(" ", "_")) for c in df_video_OT.columns]
)

df_comments_cleaned_OT = df_comments_OT.select(
    *[col(c).alias(c.lower().replace(" ", "_")) for c in df_comments_OT.columns]
)

#  Selecionar apenas as colunas com dados necessários
df_video_selected_OT = df_video_cleaned_OT.select("video_id", "title", "keyword")
df_comments_selected_OT = df_comments_cleaned_OT.select("video_id", "comment", "likes")

#  Filtrar apenas os dados não nulos
df_video_filtered_OT = df_video_selected_OT.filter("video_id IS NOT NULL")
df_comments_filtered_OT = df_comments_selected_OT.filter("video_id IS NOT NULL")

#  Criar tabelas temporárias
df_video_filtered_OT.createOrReplaceTempView("video_table")
df_comments_filtered_OT.createOrReplaceTempView("comments_table")

# Realização do JOIN
join_video_comments_OT = spark.sql("""
    SELECT v.video_id, v.title, v.keyword, c.comment, c.likes
    FROM video_table v
    JOIN comments_table c
    ON v.video_id = c.video_id
""")
#Mostra o resultado do JOIN
join_video_comments_OT.show()

# Salvando o DataFrame no formato Parquet
join_video_comments_OT.write.option('header','true').mode("overwrite").parquet("join-videos-comments-otimizado")